# RCQAの調査

## データ読み込み と 基本的な情報

論文によると，
- q の異なり数：12063
- (q,a)の数：12591
- (q,a,d)の数：54958

実際にデータ見てみると
- q の異なり数：12055
- (q,a)の数：12086
- (q,a,d)の数：56651

In [1]:
import gzip
import json

dataset = []
with gzip.open("dataset/all-v1.0.json.gz", "rt", encoding="utf-8") as fp:
    for line in fp:
        data = json.loads(line)
        if data["documents"]:
            dataset.append(data)

In [2]:
len(dataset)

12086

In [3]:
sum = 0
for i in range(len(dataset)):
    sum = sum + len(dataset[i]["documents"])
print(sum)

56651


In [9]:
dataset[len(dataset)-1] # qidが歯抜けなことが分かる

{'qid': 12591,
 'competition': 'EQIDEN_2010',
 'timestamp': '2010/03/21',
 'format': '早押し',
 'question': '主に中華料理の点心などで用いられる、竹や木を編んで作られる蒸し専用の調理器具は何でしょう?',
 'answer': '蒸籠',
 'documents': [{'title': '蒸籠',
   'text': '蒸籠、または蒸篭（せいろ、せいろう）は、竹や木を編んで作られた蒸し料理用の調理器具。',
   'score': 4},
  {'title': '蒸籠蒸し',
   'text': '蒸籠蒸しという調理法でもっとも広く作られているものとしては饅頭がある。饅頭を含めた中華料理の点心には蒸籠で蒸すものが多く、皿に乗せた肉や魚も蒸籠で蒸す手法を使う事がある。',
   'score': 0},
  {'title': '蒸籠',
   'text': '沸騰した湯の入った鍋（中華鍋や円付鍋）などの上に置き、内部に加熱された水蒸気を通すことで食材を加熱調理する。調理するものによってさまざまな大きさのものがある。さらに蒸籠は構造によって中華蒸籠と和蒸籠に分類される。中華蒸籠は竹で編まれた底とふたが一体になっているのに対し、和蒸籠は底の部分が着脱可能なすのこになっているものが多い。その名が示すとおり、通常中華蒸籠は中華料理（主に点心）、和蒸籠は和食に用いられるが、基本的な構造や使用法は変わらないため、どちらをどちらに使っても支障はない。和蒸籠には四角いものも多い。',
   'score': 3},
  {'title': '蒸し物',
   'text': '蒸す際にはあらかじめ鍋で水を沸騰させて蒸し湯を作り、その鍋の上に加熱したい食材を入れた蒸籠を置く。湯気を絶えず充満させるため、水は常に加熱し、蒸籠には蓋をしたままにしておく。中華まん、シュウマイなどの点心はこの方法で蒸されることが多い。蒸篭の蓋は竹や木を編んだものであり湯気が抜けやすく、必要以上に湯気をこもらせて食材をびしょびしょにしてしまうことが起こりにくい特長がある。',
   'score': 3},
  {'title': '蒸籠蒸し',
   'text': '蒸籠蒸し（せいろむし）は、蒸

## qid は一意か

In [ ]:
# qid が被ってることはなさそう
a = []
for i in range(len(dataset)):
    if dataset[i]["qid"] in a:
        print(dataset[i]["qid"])
    a.append(dataset[i]["qid"])

## 一意じゃない question は？

In [4]:
q = []
cnt = 0
index = []
for i in range(len(dataset)):
    if dataset[i]["question"] in q:
        #print(dataset[i]["question"])
        cnt = cnt + 1
        index.append(i)
    q.append(dataset[i]["question"])

In [8]:
print(cnt) # 一位じゃない Q の数

31


### ということで， 12086 - 31 = 12055 が一意なq. 論文によると，これは 12063 らしい． このギャップは何？

In [6]:
# なんか一意じゃない q の例
for j in range(index[0]+1):
    if dataset[j]["question"]==dataset[index[0]]["question"]:
        print(j)

1211
3199


### qid, competition, timestamp が異なることがわかる(formatも他の例では異なると思われる)

In [20]:
# 見比べ
dataset[1211]

{'qid': 1270,
 'competition': 'abc_01',
 'timestamp': '2003/03/30',
 'format': '早押し',
 'question': '中国の思想家で、「性善説」を唱えたのは孟子ですが、「性悪説」を唱えたのは誰でしょう?',
 'answer': '荀子',
 'documents': [{'title': '性悪説',
   'text': '性悪説（せいあくせつ）とは、紀元前3世紀ごろの中国の思想家荀子が、孟子の性善説に反対して唱えた人間の本性に対する主張。「人の性は悪なり、その善なるものは偽（ぎ）なり」（『荀子』性悪篇より）から来ている。',
   'score': 4},
  {'title': '人性論',
   'text': '戦国時代、孟子は性善説を唱え、告子の性無善無悪説に対抗した。一方、荀子は性悪説を唱え、礼による後天的な修正の必要性を説いた。',
   'score': 5},
  {'title': '孟子',
   'text': 'その後、荀子（じゅんし）は性悪説を唱えたが、孟子の性善説は儒教主流派の中心概念となって多くの儒者に受け継がれた。',
   'score': 5},
  {'title': '性善説',
   'text': '孟子より数十年遅く活躍した荀子は、孟子の「性善説」を批判した。この根本には「性」に関する孟子とは異なった定義がある。荀子は「性」を自然そのままの本質と考える。これは荀子が「天」を理法的な存在、あるいは宗教的なものとして捉えず、nature的な自然として理解するからである。荀子が「性」という時、欲望も含んだものとして捉えられている。そして自然そのままの本性を「悪」とした。というのも、人の「性」とその作用である「情」を放任すると、争いなどがおこり社会的混乱を招くからだという。したがって外在する「礼」によって人を矯正・感化する必要があるのだと説いた。しかし孟子「性善説」が悪の起源について矛盾を抱えていたのと同様、荀子の「性悪説」もまた善なる「礼」が何に由来するのかという点において矛盾を内包する学説であった。',
   'score': 5},
  {'title': '儒教',
   'text': '孔子の死後、儒家は八派に分かれた。その中で孟

In [21]:
dataset[3199]

{'qid': 3361,
 'competition': 'abc_02',
 'timestamp': '2004/03/21',
 'format': '早押し',
 'question': '中国の思想家で、「性善説」を唱えたのは孟子ですが、「性悪説」を唱えたのは誰でしょう?',
 'answer': '荀子',
 'documents': [{'title': '性悪説',
   'text': '性悪説（せいあくせつ）とは、紀元前3世紀ごろの中国の思想家荀子が、孟子の性善説に反対して唱えた人間の本性に対する主張。「人の性は悪なり、その善なるものは偽（ぎ）なり」（『荀子』性悪篇より）から来ている。',
   'score': 5},
  {'title': '人性論',
   'text': '戦国時代、孟子は性善説を唱え、告子の性無善無悪説に対抗した。一方、荀子は性悪説を唱え、礼による後天的な修正の必要性を説いた。',
   'score': 5},
  {'title': '孟子',
   'text': 'その後、荀子（じゅんし）は性悪説を唱えたが、孟子の性善説は儒教主流派の中心概念となって多くの儒者に受け継がれた。',
   'score': 4},
  {'title': '性善説',
   'text': '孟子より数十年遅く活躍した荀子は、孟子の「性善説」を批判した。この根本には「性」に関する孟子とは異なった定義がある。荀子は「性」を自然そのままの本質と考える。これは荀子が「天」を理法的な存在、あるいは宗教的なものとして捉えず、nature的な自然として理解するからである。荀子が「性」という時、欲望も含んだものとして捉えられている。そして自然そのままの本性を「悪」とした。というのも、人の「性」とその作用である「情」を放任すると、争いなどがおこり社会的混乱を招くからだという。したがって外在する「礼」によって人を矯正・感化する必要があるのだと説いた。しかし孟子「性善説」が悪の起源について矛盾を抱えていたのと同様、荀子の「性悪説」もまた善なる「礼」が何に由来するのかという点において矛盾を内包する学説であった。',
   'score': 4},
  {'title': '儒教',
   'text': '孔子の死後、儒家は八派に分かれた。その中で孟

### 検索クエリqが同じなので， title,textの中身も同じ．

In [14]:
# 見比べ
a = dataset[1211]["documents"]
for i in range(len(a)):
    print(a[i]['title'])

性悪説
人性論
孟子
性善説
儒教


In [15]:
# 見比べ
a = dataset[3199]["documents"]
for i in range(len(a)):
    print(a[i]['title'])

性悪説
人性論
孟子
性善説
儒教


In [18]:
# 見比べ
a = dataset[1211]["documents"]
for i in range(len(a)):
    print(i,a[i]['text'])

0 性悪説（せいあくせつ）とは、紀元前3世紀ごろの中国の思想家荀子が、孟子の性善説に反対して唱えた人間の本性に対する主張。「人の性は悪なり、その善なるものは偽（ぎ）なり」（『荀子』性悪篇より）から来ている。
1 戦国時代、孟子は性善説を唱え、告子の性無善無悪説に対抗した。一方、荀子は性悪説を唱え、礼による後天的な修正の必要性を説いた。
2 その後、荀子（じゅんし）は性悪説を唱えたが、孟子の性善説は儒教主流派の中心概念となって多くの儒者に受け継がれた。
3 孟子より数十年遅く活躍した荀子は、孟子の「性善説」を批判した。この根本には「性」に関する孟子とは異なった定義がある。荀子は「性」を自然そのままの本質と考える。これは荀子が「天」を理法的な存在、あるいは宗教的なものとして捉えず、nature的な自然として理解するからである。荀子が「性」という時、欲望も含んだものとして捉えられている。そして自然そのままの本性を「悪」とした。というのも、人の「性」とその作用である「情」を放任すると、争いなどがおこり社会的混乱を招くからだという。したがって外在する「礼」によって人を矯正・感化する必要があるのだと説いた。しかし孟子「性善説」が悪の起源について矛盾を抱えていたのと同様、荀子の「性悪説」もまた善なる「礼」が何に由来するのかという点において矛盾を内包する学説であった。
4 孔子の死後、儒家は八派に分かれた。その中で孟軻（孟子）は性善説を唱え、孔子が最高の徳目とした仁に加え、実践が可能とされる徳目義の思想を主張し、荀況（荀子）は性悪説を唱えて礼治主義を主張した。『詩』『書』『礼』『楽』『易』『春秋』といった周の書物を六経として儒家の経典とし、その儒家的な解釈学の立場から『礼記』や『易伝』『春秋左氏伝』『春秋公羊伝』『春秋穀梁伝』といった注釈書や論文集である伝が整理された（完成は漢代）。


In [19]:
# 見比べ
a = dataset[3199]["documents"]
for i in range(len(a)):
    print(i,a[i]['text'])

0 性悪説（せいあくせつ）とは、紀元前3世紀ごろの中国の思想家荀子が、孟子の性善説に反対して唱えた人間の本性に対する主張。「人の性は悪なり、その善なるものは偽（ぎ）なり」（『荀子』性悪篇より）から来ている。
1 戦国時代、孟子は性善説を唱え、告子の性無善無悪説に対抗した。一方、荀子は性悪説を唱え、礼による後天的な修正の必要性を説いた。
2 その後、荀子（じゅんし）は性悪説を唱えたが、孟子の性善説は儒教主流派の中心概念となって多くの儒者に受け継がれた。
3 孟子より数十年遅く活躍した荀子は、孟子の「性善説」を批判した。この根本には「性」に関する孟子とは異なった定義がある。荀子は「性」を自然そのままの本質と考える。これは荀子が「天」を理法的な存在、あるいは宗教的なものとして捉えず、nature的な自然として理解するからである。荀子が「性」という時、欲望も含んだものとして捉えられている。そして自然そのままの本性を「悪」とした。というのも、人の「性」とその作用である「情」を放任すると、争いなどがおこり社会的混乱を招くからだという。したがって外在する「礼」によって人を矯正・感化する必要があるのだと説いた。しかし孟子「性善説」が悪の起源について矛盾を抱えていたのと同様、荀子の「性悪説」もまた善なる「礼」が何に由来するのかという点において矛盾を内包する学説であった。
4 孔子の死後、儒家は八派に分かれた。その中で孟軻（孟子）は性善説を唱え、孔子が最高の徳目とした仁に加え、実践が可能とされる徳目義の思想を主張し、荀況（荀子）は性悪説を唱えて礼治主義を主張した。『詩』『書』『礼』『楽』『易』『春秋』といった周の書物を六経として儒家の経典とし、その儒家的な解釈学の立場から『礼記』や『易伝』『春秋左氏伝』『春秋公羊伝』『春秋穀梁伝』といった注釈書や論文集である伝が整理された（完成は漢代）。
